In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

from delfi.utils.viz import plot_pdf
from tqdm import tqdm_notebook as tqdm
from parameters import ParameterSet

import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import delfi.kernel as dk
from delfi.simulator.BaseSimulator import BaseSimulator
import delfi.utils.viz as dv


In [ ]:
params = ParameterSet({})

params.seed = 239

params.infer = ParameterSet({})
params.infer.n_components = 1
params.infer.n_hiddens = [10, 10]
params.infer.pilot_samples = 100
params.infer.svi = False
params.infer.prior_norm = True

params.run = ParameterSet({})
params.run.n_train = 50000
#params.run.minibatch = 100
params.run.epochs = 20
#params.run.lr = 0.02

fontsize=20

In [ ]:
prior_mean = 2
prior_var = 0.5

prop_prior_mean=1.09090909
prop_prior_var=0.11615753

noise_var = 0.01

gt = np.array([ 0.6 ])

In [ ]:
def gauss(x, mean, cov):
    covi = np.linalg.inv(cov)
    dots = np.einsum('...i,ij,...j', x - mean, covi, x - mean)
    ret = 1 / np.sqrt(2 * np.pi * np.linalg.det(cov)) * np.exp(-0.5 * dots)
    assert ret.shape == x.shape[:-1]
    return ret

class ForwardModel(BaseSimulator):
    def __init__(self, seed=None):
        super().__init__(dim_param=1, seed=seed)
        self.noise_std = np.sqrt(noise_var)
    
    def gen_single(self, params):
        means = self.get_means(params)
        sample = means + self.noise_std * self.rng.normal(size=params.shape)
        return { "data" :  np.array(sample) }
    
    def likelihood(self, x, params):
        means = self.get_means(params)
        diffs = means - x
        ret = gauss(diffs.reshape(-1,1), [0], (self.noise_std ** 2) * np.eye(1))
        return ret
        
    def get_means(self, params):
        return np.sign(params) * (np.abs(params) ** (1/3))
    
    def get_mle(self, params):
        return np.asarray(params) ** 3

In [ ]:
def init_g(seed):
    

    m = ForwardModel(seed=seed)
    prior = dd.Gaussian(m=[prior_mean], S=[[prior_var]], seed=seed+1)
    prop_prior = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=seed+2)
    s = ds.Identity(1)

    return dg.Default(m, prior, s, seed=seed+3)

g = init_g(params.seed)
xobs = g.model.get_means(gt)

params.run.n_train = 100000
params.run.epochs = 200
res = infer.SNPE(generator=g, obs=[xobs], verbose=True, **params.infer)

logs, tds, posteriors = res.run(n_rounds=1, **params.run)

posterior = posteriors[-1]


In [ ]:
g = init_g(params.seed)
g.prior = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=params.seed+1234)

params.run.n_train = 100000
params.run.epochs = 200
res_uncorr = infer.SNPE(generator=g, obs=[xobs], verbose=True, **params.infer)

logs, tds, posteriors_uncorr = res_uncorr.run(n_rounds=1, **params.run)


posterior_uncorr = posteriors_uncorr[-1]


In [ ]:
g = init_g(params.seed)
proposal = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=params.seed+1234)

res_SNPE = infer.SNPE(generator=g, obs=[xobs], verbose=True, convert_to_T=None, **params.infer)

params.run.n_train = 100000
params.run.epochs = 200

logs, tds, posteriors_SNPE = res_SNPE.run(n_rounds=1, proposal = proposal, **params.run)

posterior_SNPE = posteriors_SNPE[-1]


In [ ]:
fig = plt.figure(figsize=(9,9))

###############################################################################
# sampling from proposal prior and running forward model
n_samp = 500
np.random.seed(44)

g = init_g(params.seed)
g.proposal = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=params.seed)
theta_samp = (np.sqrt(prop_prior_var) * np.random.randn(n_samp) + prop_prior_mean).reshape(-1,1)
x_samp = np.sqrt(noise_var) * np.random.randn(n_samp) + g.model.get_means(theta_samp).flatten()
#theta_samp = (tds[-1][0].reshape(-1,1) * res_SNPE.params_std) + res_SNPE.params_mean
#x_samp     = (tds[-1][1].reshape(-1,1) * res_SNPE.stats_std)  + res_SNPE.stats_mean

theta_lims = [0, 3]
thetas = np.linspace(theta_lims[0], theta_lims[1], 500)

xlims = [0, g.model.get_means(theta_lims[1]) * 1.1]

xs = np.linspace(xlims[0], xlims[1], 100)

posts = [ res.predict([[x]]) for x in xs ]
post_means = np.array([ post.xs[0].m[0] for post in posts ])
post_vars = np.array([ post.xs[0].S[0,0] for post in posts ])

posts_SNPE = [ res_SNPE.predict([[x]]) for x in xs ]
post_means_SNPE = np.array([ post.xs[0].m[0] for post in posts_SNPE ])
post_vars_SNPE = np.array([ post.xs[0].S[0,0] for post in posts_SNPE ])

gs = gridspec.GridSpec(2, 3, width_ratios=[1, 3, 1], height_ratios=[4,1])

prior_col = "blue"
prop_prior_col = "magenta"
post_col = "blue"
post_uncorr_col = "magenta"

# LEFTHAND SIDE
axl = plt.subplot(gs[0,0])
#plt.plot(prop_prior, yy, '--',color='0.55', label='proposal prior')
plt.plot(g.prior.eval(thetas.reshape(-1,1), log=False), thetas, '--', c=prior_col, label='prior')
plt.plot(g.proposal.eval(thetas.reshape(-1,1), log=False), thetas, c=prop_prior_col, label='proposal prior')

axl.set_ylim(theta_lims)
axl.set_yticks([])
axl.set_xticks([])
plt.ylabel(r'$\theta$', fontsize=fontsize)
axl.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),  ncol=1, fontsize=fontsize)
axl.spines['top'].set_visible(False)
axl.spines['right'].set_visible(False)
axr = plt.subplot(gs[0,2])



# RIGHTHAND SIDE
plt.plot(posterior.eval(thetas.reshape(-1,1), log=False), thetas, c=post_col, label='posterior')
plt.plot(posterior_uncorr.eval(thetas.reshape(-1,1), log=False), thetas, 
         alpha=0.5, c=post_uncorr_col, label='uncorrected posterior \n(posterior for proposal)')
#    plt.plot(pp_post,yy,color=col_2, label='posterior')
#plt.plot(pp_param,yy,color=col_3, label='parametrised posterior')
axr.set_ylim(theta_lims)
#ax.set_xticks([])
axr.set_xticks([])
plt.ylabel('')
axr.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=1, fontsize=fontsize)
#plt.legend(bbox_to_anchor=(1.25, 1), loc='upper right')


hl, ll = axl.get_legend_handles_labels()
hr, lr = axr.get_legend_handles_labels()
axr.set_yticks([])
axr.spines['top'].set_visible(False)
axr.spines['right'].set_visible(False)

axm = plt.subplot(gs[0,1])
plt.ylabel('')

iws = np.exp(g.prior.eval(theta_samp, log=True) - g.proposal.eval(theta_samp, log=True))

# Draw true posterior
plt.scatter(x_samp, theta_samp.flatten(), color='0.05', s=30 * np.log((1.1 - np.min(iws)) + iws), alpha=0.4)

# Draw posterior means (in orange)
std_col = plt.get_cmap("tab10")(0)

std_col = 'gray'

plt.plot(xs,post_means, c=std_col, label='posterior', alpha=0.5)
plt.plot(xs,post_means + 2 * np.sqrt(post_vars),ls='--', c=std_col, alpha=0.5)
plt.plot(xs,post_means - 2 * np.sqrt(post_vars),ls='--', c=std_col, alpha=0.5)

plt.fill_between(xs,post_means-2 * np.sqrt(post_vars),
                 post_means+2 * np.sqrt(post_vars), facecolor=std_col,
                 alpha=0.15)


std_col = prop_prior_col

plt.plot(xs,post_means_SNPE, c=std_col, label='SNPE')
plt.plot(xs,post_means_SNPE + 2 * np.sqrt(post_vars_SNPE),ls='--', c=std_col)
plt.plot(xs,post_means_SNPE - 2 * np.sqrt(post_vars_SNPE),ls='--', c=std_col)

plt.fill_between(xs,post_means_SNPE-2 * np.sqrt(post_vars_SNPE),
                 post_means_SNPE+2 * np.sqrt(post_vars_SNPE), facecolor=std_col,
                 alpha=0.1)

# Draw vertical lines around chosen point
axm.axvline(xobs,lw=2, c='k')
#     axm.vlines(obs_stats-eps, 0, axm.get_ylim()[1], lw=1, color='0.25')
#     axm.vlines(obs_stats+eps, 0, axm.get_ylim()[1], lw=1, color='0.25')

axm.set_xlim(xlims)
axm.set_ylim(theta_lims)
axm.set_xticks([])
axm.set_yticks([])
#plt.xlabel('x')
#plt.legend(bbox_to_anchor=(0.0, 1), loc='upper left')

#plt.xticks([xobs], [r'$x_{o}$'])

#axm.set_xlabel("$x$", fontsize=fontsize)
axm.spines['top'].set_visible(False)
axm.spines['right'].set_visible(False)

axb = plt.subplot(gs[1,1])


def compute_marginals(prop):
    thetas = prop.gen(100000)
    means = g.model.get_means(thetas)
    
    ret = means + np.sqrt(noise_var) * np.random.normal(size=thetas.shape)
    return ret
    
prior_marg_samples = compute_marginals(g.prior)
prop_prior_marg_samples = compute_marginals(g.proposal)

from sklearn.neighbors.kde import KernelDensity

prior_marg_density = KernelDensity(kernel='gaussian', 
                                   bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prior_marg_samples)
prop_prior_marg_density = KernelDensity(kernel='gaussian', 
                                       bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prop_prior_marg_samples)

plt.plot(xs, np.exp(prior_marg_density.score_samples(xs.reshape(-1,1))), '--', c=prior_col, label='prior')
plt.plot(xs, np.exp(prop_prior_marg_density.score_samples(xs.reshape(-1,1))), c=prop_prior_col, label='prioposal prior')

#plt.plot(xs, gen_mod.eval(xs.reshape(-1,1), log=False), color='0.0', label='marginal')

axb.set_xlim(xlims)
axb.set_xticks([])
plt.xticks([xobs], [r'$x_{o}$'], fontsize=fontsize)
axb.set_yticks([])

axb.set_xlabel("$x$", fontsize=fontsize)
axb.set_ylabel("density", fontsize=fontsize)
axb.spines['top'].set_visible(False)
axb.spines['right'].set_visible(False)
#axb.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
 #          fancybox=True, shadow=True, ncol=1)
axb.axvline(xobs, lw=2, c='k')


plt.tight_layout()

plt.savefig('CDE_with_proposal.pdf')

plt.show()

In [ ]:
fig = plt.figure(figsize=(9,9))

###############################################################################
# sampling from proposal prior and running forward model
n_samp = 1000
np.random.seed(44)

g = init_g(params.seed)
g.proposal = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=params.seed)
theta_samp = (np.sqrt(prior_var) * np.random.randn(n_samp) + prior_mean).reshape(-1,1)
x_samp = np.sqrt(noise_var) * np.random.randn(n_samp) + g.model.get_means(theta_samp).flatten()
#theta_samp = (tds[-1][0].reshape(-1,1) * res_SNPE.params_std) + res_SNPE.params_mean
#x_samp     = (tds[-1][1].reshape(-1,1) * res_SNPE.stats_std)  + res_SNPE.stats_mean

theta_lims = [0, 3]
thetas = np.linspace(theta_lims[0], theta_lims[1], 500)

xlims = [0, g.model.get_means(theta_lims[1]) * 1.1]

xs = np.linspace(xlims[0], xlims[1], 100)

posts = [ res.predict([[x]]) for x in xs ]
post_means = np.array([ post.xs[0].m[0] for post in posts ])
post_vars = np.array([ post.xs[0].S[0,0] for post in posts ])

posts_SNPE = [ res_SNPE.predict([[x]]) for x in xs ]
post_means_SNPE = np.array([ post.xs[0].m[0] for post in posts_SNPE ])
post_vars_SNPE = np.array([ post.xs[0].S[0,0] for post in posts_SNPE ])

gs = gridspec.GridSpec(2, 3, width_ratios=[1, 3, 1], height_ratios=[4,1])

prior_col = "blue"
prop_prior_col = "magenta"
post_col = "blue"
post_uncorr_col = "magenta"

# LEFTHAND SIDE
axl = plt.subplot(gs[0,0])
plt.plot(g.prior.eval(thetas.reshape(-1,1), log=False), thetas, '--', c=prior_col, label='prior')
axl.set_ylim(theta_lims)
axl.set_xlim(xlims)
axl.set_yticks([])
axl.set_xticks([])
axl.spines['top'].set_visible(False)
axl.spines['right'].set_visible(False)
plt.ylabel(r'$\theta$', fontsize=fontsize)
plt.xlabel('prior', fontsize=fontsize)

axr = plt.subplot(gs[0,2])

# RIGHTHAND SIDE
plt.plot(posterior.eval(thetas.reshape(-1,1), log=False), thetas, c=post_col, label='posterior')
axr.set_ylim(theta_lims)
axr.set_xticks([])
axr.spines['top'].set_visible(False)
axr.spines['right'].set_visible(False)
plt.ylabel('')
plt.xlabel('posterior', fontsize=fontsize)

hl, ll = axl.get_legend_handles_labels()
hr, lr = axr.get_legend_handles_labels()
axr.set_yticks([])

axm = plt.subplot(gs[0,1])
plt.ylabel('')

iws = np.exp(g.prior.eval(theta_samp, log=True) - g.proposal.eval(theta_samp, log=True))

# Draw true posterior
plt.scatter(x_samp, theta_samp.flatten(), color='0.05', s=5,alpha=0.4)

# Draw posterior means (in orange)
std_col = plt.get_cmap("tab10")(0)

std_col = 'gray'

plt.plot(xs,post_means, c=std_col, label='posterior', alpha=0.5)
plt.plot(xs,post_means + 2 * np.sqrt(post_vars),ls='--', c=std_col, alpha=0.5)
plt.plot(xs,post_means - 2 * np.sqrt(post_vars),ls='--', c=std_col, alpha=0.5)

plt.fill_between(xs,post_means-2 * np.sqrt(post_vars),
                 post_means+2 * np.sqrt(post_vars), facecolor=std_col,
                 alpha=0.15)


# Draw vertical lines around chosen point
axm.axvline(xobs,lw=2, c='k')


axm.set_xlim(xlims)
axm.set_ylim(theta_lims)
axm.set_xticks([])
axm.set_yticks([])
axm.spines['top'].set_visible(False)
axm.spines['right'].set_visible(False)

axm.set_xlabel("$x$", fontsize=fontsize)
plt.xticks([xobs], [r'$x_{o}$'], fontsize=fontsize)

plt.tight_layout()

plt.savefig('CDE_with_prior.pdf')

plt.show()

# figures on different proposals

In [ ]:
path = '/home/marcel/Dropbox (mackelab)/team/Write/Posters/2018_Edinburgh/figures/col1/'
from scipy.misc import logsumexp

N = 1000
N_show = 100

seed = 47

fig = plt.figure(figsize=(11.25,9))
gs = gridspec.GridSpec(2, 3, width_ratios=[1,3,3], height_ratios=[4,1])

from sklearn.neighbors.kde import KernelDensity

def compute_marginals(prop, prior=None, N=1000):
    thetas = prop.gen(N)
    
    means = g.model.get_means(thetas)
    
    ret = means + np.sqrt(noise_var) * np.random.normal(size=thetas.shape)
    
    if prior is None:
        return ret
    else:
        iws = prior.eval(thetas, log=True) - prop.eval(thetas, log=True)
        iws = np.exp( iws - logsumexp(iws) ) 
        return ret, iws

def init_g(seed):
    

    m = ForwardModel(seed=seed)
    prior = dd.Gaussian(m=[prior_mean], S=[[prior_var]], seed=seed+1)
    prop_prior = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=seed+2)
    s = ds.Identity(1)

    return dg.Default(m, prior, s, seed=seed+3)

p_true = dd.Gaussian(m=0.8*np.ones(1), S=0.05*np.eye(1), seed = seed)
obs_stats = 0.84343267 * np.ones((1,1))

sig2s = np.linspace(np.sqrt(0.05), np.sqrt(0.5), 100)**2
sig2s = sig2s[::-1][[25, 50, 75, 99]]
ms    = np.linspace(0.8, 2., 100)
ms    = ms[::-1][[25, 50, 75, 99]]


clrs = ['g', 'c',  'm', 'r']
linewidth=3.

theta_lims = [0, 3]
xx = np.linspace(theta_lims[0], theta_lims[1], 500).reshape(-1,1)
g = init_g(seed)
xlims = [0, g.model.get_means(theta_lims[1]) * 1.1]


ax = plt.subplot(gs[0,1])
std_col = 'gray'
ax.plot(xs,post_means, c=std_col, label='posterior')
ax.plot(xs,post_means + 2 * np.sqrt(post_vars),ls='--', c=std_col)
ax.plot(xs,post_means - 2 * np.sqrt(post_vars),ls='--', c=std_col)
plt.fill_between(xs,post_means-2 * np.sqrt(post_vars),
                 post_means+2 * np.sqrt(post_vars), facecolor=std_col,
                 alpha=0.15)


for i in range(len(sig2s)):
    
    sig2 = sig2s[i]
    m = ms[i]
    p_prop = dd.Gaussian(m=m*np.ones(1), S = sig2*np.eye(1), seed=seed+1234+i)

    g = init_g(seed)
    g.proposal = p_prop
    trn_data = g.gen(N)

    print('Var[th] = ', trn_data[0].var())

    ax = plt.subplot(gs[0,0])
    ax.plot(  p_prop.eval(xx, log=False), xx, '-', color=clrs[i], linewidth=linewidth)

    ax = plt.subplot(gs[0,1])
    x_samp, theta_samp = trn_data[1][:N_show], trn_data[0][:N_show]
    #ax.plot( x_samp, theta_samp, '.', color=clrs[i])    
    iws = np.exp(g.prior.eval(theta_samp, log=True) - g.proposal.eval(theta_samp, log=True))
    iws /= iws.sum()
    ax.scatter(x_samp, theta_samp.flatten(), color=clrs[i], s=30 * np.log((1.1 - np.min(iws)) + 50*iws), alpha=0.4)
    
    liws = (g.prior.eval(trn_data[0]) - g.proposal.eval(trn_data[0]))
    iws = np.exp(liws - logsumexp(liws))
        
    prop_prior_marg_samples, marg_iws = compute_marginals(g.proposal, g.prior)
    prop_prior_marg_density = KernelDensity(kernel='gaussian', 
                                           bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prop_prior_marg_samples)
    iw_marg_density = KernelDensity(kernel='gaussian', 
                                    bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prop_prior_marg_samples, 
                                                                                sample_weight=marg_iws)
    ax = plt.subplot(gs[1,1])
    ax.plot(xs, np.exp(iw_marg_density.score_samples(xs.reshape(-1,1))),  color=clrs[i], 
            label='importance weighted', linewidth=linewidth)    
    ax = plt.subplot(gs[1,2])
    ax.plot(xs, np.exp(prop_prior_marg_density.score_samples(xs.reshape(-1,1))), 
            c=clrs[i], alpha=0.5, label='proposal prior', linewidth=linewidth/2)
    
    
ax = plt.subplot(gs[0,0])
#ax.plot( p_true.eval(xx, log=False), xx, '-', color='k')
ax.plot( g.prior.eval(xx, log=False), xx, '--', color='b', linewidth=linewidth)
plt.ylabel(r'$\theta$', fontsize=fontsize)
plt.xlabel('density', fontsize=fontsize)
#plt.box('off')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.yticks([])
plt.xticks([])
ax.set_ylim(theta_lims)

ax = plt.subplot(gs[0,1])
#plt.box('off')
plt.xticks([xobs], [r'$x_{o}$'], fontsize=fontsize)
plt.yticks([])
ax.set_ylim(theta_lims)
ax.set_xlim(xlims)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.axvline(xobs, lw=2, c='k')


ax = plt.subplot(gs[1,1])
prior_marg_samples = compute_marginals(g.prior)
prior_marg_density = KernelDensity(kernel='gaussian', 
                                   bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prior_marg_samples)
ax.plot(xs, np.exp(prior_marg_density.score_samples(xs.reshape(-1,1))), '--', c='b', label='prior', linewidth=3)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim(xlims)
plt.xticks([])
plt.yticks([])
ax.axvline(xobs, lw=2, c='k')
plt.ylabel('density', fontsize=fontsize)
plt.xlabel("$x$", fontsize=fontsize)
plt.xticks([xobs], [r'$x_{o}$'], fontsize=fontsize)

ax = plt.subplot(gs[1,2])
prior_marg_samples = compute_marginals(g.prior)
prior_marg_density = KernelDensity(kernel='gaussian', 
                                   bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prior_marg_samples)
ax.plot(xs, np.exp(prior_marg_density.score_samples(xs.reshape(-1,1))), '--', c='b', label='prior', linewidth=3)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlim(xlims)
plt.xticks([])
plt.yticks([])
ax.axvline(xobs, lw=2, c='k')
plt.ylabel('density', fontsize=fontsize)
plt.ylabel('density', fontsize=fontsize)
plt.xlabel("$x$", fontsize=fontsize)
plt.xticks([xobs], [r'$x_{o}$'], fontsize=fontsize)


plt.tight_layout()

plt.savefig(path + 'narrowness_vs_ess.pdf')

plt.show()

In [ ]:
path = '/home/marcel/Dropbox (mackelab)/team/Write/Posters/2018_Edinburgh/figures/col1/'
from scipy.misc import logsumexp

N = 1000
N_show = 100

seed = 47

fig = plt.figure(figsize=(4.5, 4.5))
gs = gridspec.GridSpec(1,1, width_ratios=[1], height_ratios=[1])

from sklearn.neighbors.kde import KernelDensity

def compute_marginals(prop, prior=None, N=10000):
    thetas = prop.gen(N)
    
    means = g.model.get_means(thetas)
    
    ret = means + np.sqrt(noise_var) * np.random.normal(size=thetas.shape)
    
    if prior is None:
        return ret
    else:
        iws = prior.eval(thetas, log=True) - prop.eval(thetas, log=True)
        iws = np.exp( iws - logsumexp(iws) ) 
        return ret, iws

def init_g(seed):
    

    m = ForwardModel(seed=seed)
    prior = dd.Gaussian(m=[prior_mean], S=[[prior_var]], seed=seed+1)
    prop_prior = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=seed+2)
    s = ds.Identity(1)

    return dg.Default(m, prior, s, seed=seed+3)

p_true = dd.Gaussian(m=0.8*np.ones(1), S=0.05*np.eye(1), seed = seed)
obs_stats = 0.84343267 * np.ones((1,1))

sig2s = np.linspace(np.sqrt(0.05), np.sqrt(0.5), 100)**2
sig2s = sig2s[::-1][[25, 50, 75, 99]]
ms    = np.linspace(0.8, 2., 100)
ms    = ms[::-1][[25, 50, 75, 99]]


clrs = ['g', 'c',  'm', 'r']
linewidth=3.

theta_lims = [0, 3]
xx = np.linspace(theta_lims[0], theta_lims[1], 500).reshape(-1,1)
g = init_g(seed)
xlims = [0, g.model.get_means(theta_lims[1]) * 1.1]


ax = plt.subplot(gs[0,0])
ax.plot([1,N],[0,1], 'b--', label='prior', linewidth=linewidth)


for i in range(len(sig2s)):
    
    sig2 = sig2s[i]
    m = ms[i]
    p_prop = dd.Gaussian(m=m*np.ones(1), S = sig2*np.eye(1), seed=seed+1234+i)

    g = init_g(seed)
    g.proposal = p_prop
    trn_data = g.gen(N)

    print('Var[th] = ', trn_data[0].var())
    
    ax = plt.subplot(gs[0,0])    
    liws = (g.prior.eval(trn_data[0]) - g.proposal.eval(trn_data[0]))
    iws = np.exp(liws - logsumexp(liws))
    ax.plot(np.arange(iws.size)+1, np.cumsum(np.sort(iws)[::-1]), '-', color=clrs[i], linewidth=linewidth)
    
    ESS = 1./np.sum( (iws/iws.sum())**2 )
    
    if ESS > 100:
        plt.text(N/2, (len(sig2s)-i-1)*0.1, str(np.round(ESS*10)/(10)), color=clrs[i], fontsize=fontsize)
    else:
        plt.text(N/2, (len(sig2s)-i-1)*0.1, str(np.round(ESS*100)/(100)), color=clrs[i], fontsize=fontsize)
            

ax = plt.subplot(gs[0,0])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xticks([1, N], ['1', 'N'], fontsize=fontsize)
plt.yticks([0, 0.5, 1.], fontsize=fontsize)
plt.text(N/2, (len(sig2s))*0.1, 'ESS (out of N='+str(N)+'):', color='k', fontsize=fontsize)
plt.xlabel('number of samples (sorted)', fontsize=fontsize)
plt.ylabel('cumulative importance weights', fontsize=fontsize)


plt.tight_layout()

plt.savefig(path + 'narrowness_vs_ess_inset.pdf')

plt.show()

In [ ]:
import abc
from delfi.utils.meta import ABCMetaDoc
class My_Helper_Kernel(metaclass=ABCMetaDoc):
    def __init__(self, obs, A, Z):
        """Temporary substitute for working Gaussian kernel class in delfi
        Parameters
        ----------
        obs : 1 x dim
            center of kernel
        A : dim x dim or dim
            kernel matrix (cf. precision matrix for Gaussian distributions)
        Z : kernel log-normalizer
        -----
        Notes:
        k(x) = exp( - (x-obs)' A (x-obs) + Z )
        """

        assert obs.shape[0] == 1, 'obs.shape[0] must be 1'
        assert obs.shape[1] >= 1, 'obs.shape[1] must be >= 1'

        self.dim = obs.shape[1]
        self.obs = obs

        self.diag_A = False if A.ndim>1 else True        
        self.A = A
        self.Z = Z

        if self.diag_A:
            self.B = np.sqrt(A)
        else:
            self.B = np.linalg.cholesky(A)

    @abc.abstractmethod
    def kernel(u):
        pass        


    def eval(self, x):
        """Kernel for loss calibration

        Parameters
        ----------
        x : N x dim
            points at which to evaluate kernel

        Returns
        -------
        weights : N x 1
            normalized to be 1. for x = obs
        """
        assert x.shape[0] >= 1, 'x.shape[0] needs to be >= 1'
        assert x.shape[1] == self.dim, 'x.shape[1] needs to be == self.obs'

        if self.diag_A:
            z = (x-self.obs) * self.B.reshape(1,-1)
            out = np.exp( -(z*z).sum(axis=1) + self.Z)
        else:
            z = (x-self.obs).dot(self.B)
            out = np.exp( -(z*z).sum(axis=1) + self.Z)

        return out

In [ ]:
from scipy.misc import logsumexp
from delfi.kernel.Kernel_learning import kernel_opt


path = '/home/marcel/Dropbox (mackelab)/team/Write/Posters/2018_Edinburgh/figures/col3/'
N = 1000
N_show = 1000


gs = gridspec.GridSpec(1, 4, width_ratios=[2,4,5,4], height_ratios=[1])

def init_g(seed):
    

    m = ForwardModel(seed=seed)
    prior = dd.Gaussian(m=[prior_mean], S=[[prior_var]], seed=seed+1)
    prop_prior = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=seed+2)
    s = ds.Identity(1)

    return dg.Default(m, prior, s, seed=seed+3)

p_true = dd.Gaussian(m=0.8*np.ones(1), S=0.05*np.eye(1), seed = seed)
obs_stats = 0.84343267 * np.ones((1,1))


sig2s = np.linspace(np.sqrt(0.05), np.sqrt(0.5), 100)**2
sig2s = sig2s[::-1][[25, 50, 75, 99]]
ms    = np.linspace(0.8, 2., 100)
ms    = ms[::-1][[25, 50, 75, 99]]

taus = np.linspace(0.001, 0.8, 801)

seed = 47

fig = plt.figure(figsize=(18,7.5))

#ms, sig2s = [ms[-1]], [sig2s[-1]]

clrs = ['g', 'c',  'm', 'r']

theta_lims = [0, 3]
xx = np.linspace(theta_lims[0], theta_lims[1], 500).reshape(-1,1)
g = init_g(seed)
xlims = [0, g.model.get_means(theta_lims[1]) * 1.1]

for i in range(len(sig2s)):

    sig2 = sig2s[i]
    m = ms[i]
    p_prop = dd.Gaussian(m=m*np.ones(1), S = sig2*np.eye(1), seed=seed+1234+i)

    g = init_g(seed)
    g.proposal = p_prop
    trn_data = g.gen(N)

    print('Var[th] = ', trn_data[0].var())

    ax = plt.subplot(gs[0,0])
    ax.plot(  p_prop.eval(xx, log=False), xx, '-', color=clrs[i], linewidth=linewidth)

    ax = plt.subplot(gs[0,1])

    liws = (g.prior.eval(trn_data[0]) - g.proposal.eval(trn_data[0]))
    iws = np.exp(liws - logsumexp(liws))    

    ESS = 1./np.sum( (iws/iws.sum())**2 )

    dists = np.sqrt( np.sum( (trn_data[1] - obs_stats)**2, axis=1) )
    ax.plot( dists[:N_show], liws[:N_show], 'o', color=clrs[i])
    plt.xlabel("$|| x - x_o ||$", fontsize=fontsize)
    plt.ylabel('log importance weights', fontsize=fontsize)

    kESS = np.zeros(taus.size)
    kcbl = np.zeros(taus.size)
    for j in range(len(taus)): 

        cbk = My_Helper_Kernel(obs=obs_stats, Z=np.zeros(1), A=(1./taus[j])**2*np.ones(1)).eval(trn_data[1])
        kiws = iws * cbk
        kESS[j] = 1./np.sum( (kiws/kiws.sum())**2 )
        
        kcbl[j] = - np.mean(np.log(cbk)) + np.log(np.mean(kiws))

    cbkrnl, cbl = kernel_opt(
        iws=iws.astype(np.float32), 
        stats=trn_data[1].reshape(-1,1).astype(np.float32),
        obs=obs_stats.reshape(1,1).astype(np.float32), 
        kernel_loss='x_kl', 
        epochs=10000, #epochs 
        minibatch=1000, #minibatch, 
        stop_on_nan=False,
        seed=42, 
        monitor=None)
        
    ax = plt.subplot(gs[0,2])
    ax.plot(taus, kESS, color=clrs[i], linewidth=3.0)
    ax.plot([0.91, 0.93], [ESS, ESS], color=clrs[i], linewidth=3.0)

    txt_offsets = [-5, -5, -5, -5]

    if ESS > 100:
        ax.text(0.95, ESS+txt_offsets[i], str(np.round(ESS*10)/(10)), color=clrs[i], fontsize=fontsize)
    else:
        ax.text(0.95, ESS+txt_offsets[i], str(np.round(ESS*100)/(100)), color=clrs[i], fontsize=fontsize)



    tau_est = 1./cbkrnl.B
    
    if i == 2:
        tau_plot = tau_est
    
    idx = np.argmin(np.abs(tau_est - taus))

    ax.plot(tau_est, kESS[idx]+10, 'v', color=clrs[i], markersize=12, markerfacecolor=clrs[i],
             markeredgewidth=1., markeredgecolor='k')

    ax = plt.subplot(gs[0,3])    
    ax.plot(taus, kcbl, color=clrs[i], linewidth=3.0)
    ax.plot(tau_est, kcbl[idx]+0.015, 'v', color=clrs[i], markersize=12, markerfacecolor=clrs[i],
             markeredgewidth=1., markeredgecolor='k')
    
    
ax = plt.subplot(gs[0,0])
ax.plot( g.prior.eval(xx, log=False), xx, '--', color='b')
plt.ylabel(r'$\theta$', fontsize=fontsize)
plt.xlabel('density', fontsize=fontsize)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.yticks([])
plt.xticks([])
ax.set_ylim(theta_lims)

ax = plt.subplot(gs[0,1])
plt.yticks([])
plt.xticks([0., 0.2, 0.4, 0.6, 0.8], fontsize=fontsize)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.axis([-0.01, 0.81, np.min(liws[:N_show])*1.01, np.max(liws[:N_show])*1.01])

xx = np.linspace(0, 0.8, 801).reshape(-1,1)
cbk = My_Helper_Kernel(obs=0*obs_stats, Z=np.ones(1), A=(1./tau_plot)**2*np.ones(1)).eval(xx)
ax.plot(xx, np.log(cbk), 'k', linewidth=linewidth+0.5)
ax.plot(xx, np.log(cbk), 'm', linewidth=linewidth-0.5)

ax = plt.subplot(gs[0,2])
plt.xticks([0., 0.2, 0.4, 0.6, 0.8, 0.92], 
           ['0', '0.2', '0.4', '0.6', '0.8', r'$\infty$'], fontsize=fontsize)
plt.yticks(np.array([0., 0.2, 0.4, 0.6, 0.8, 1.0])*N, fontsize=fontsize)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlabel('kernel bandwidth ' + r'$\tau$', fontsize=fontsize)
plt.ylabel('effective sample size', fontsize=fontsize)
plt.axis([-0.01, 1.06, 0, N+1])

ax = plt.subplot(gs[0,3])
plt.xticks([0., 0.2, 0.4, 0.6, 0.8], 
           ['0', '0.2', '0.4', '0.6', '0.8'], fontsize=fontsize)
plt.yticks(np.array([-7.4, -7., -6.6, -6.2]), fontsize=fontsize)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlabel('kernel bandwidth ' + r'$\tau$', fontsize=fontsize)
plt.ylabel(r'$\mathcal{L}(\tau)$', fontsize=fontsize)
plt.axis([-0.01, 0.81, -7.5, -6])

plt.tight_layout()

plt.savefig(path + 'importance_weights_vs_distance.pdf')

plt.show()

In [ ]:
from scipy.misc import logsumexp
from delfi.kernel.Kernel_learning import kernel_opt


path = '/home/marcel/Dropbox (mackelab)/team/Write/Posters/2018_Edinburgh/figures/col3/'
N = 1000
N_show = 1000


fig = plt.figure(figsize=(9,2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1], height_ratios=[1])

def init_g(seed):
    

    m = ForwardModel(seed=seed)
    prior = dd.Gaussian(m=[prior_mean], S=[[prior_var]], seed=seed+1)
    prop_prior = dd.Gaussian(m=[prop_prior_mean], S=[[prop_prior_var]], seed=seed+2)
    s = ds.Identity(1)

    return dg.Default(m, prior, s, seed=seed+3)


p_true = dd.Gaussian(m=0.8*np.ones(1), S=0.05*np.eye(1), seed = seed)
obs_stats = 0.84343267 * np.ones((1,1))


sig2s = np.linspace(np.sqrt(0.05), np.sqrt(0.5), 100)**2
sig2s = sig2s[::-1][[25, 50, 75, 99]]
ms    = np.linspace(0.8, 2., 100)
ms    = ms[::-1][[25, 50, 75, 99]]

taus = np.linspace(0.001, 0.8, 801)

seed = 47


#ms, sig2s = [ms[-1]], [sig2s[-1]]

clrs = ['g', 'c',  'm', 'r']

theta_lims = [0, 3]
xx = np.linspace(xlims[0], xlims[1], 500).reshape(-1,1)
g = init_g(seed)
xlims = [0, g.model.get_means(theta_lims[1]) * 1.1]

for i in range(len(sig2s)):

    sig2 = sig2s[i]
    m = ms[i]
    p_prop = dd.Gaussian(m=m*np.ones(1), S = sig2*np.eye(1), seed=seed+1234+i)

    g = init_g(seed)
    g.proposal = p_prop
    trn_data = g.gen(N)

    print('Var[th] = ', trn_data[0].var())

    ax = plt.subplot(gs[0,0])

    liws = (g.prior.eval(trn_data[0]) - g.proposal.eval(trn_data[0]))
    iws = np.exp(liws - logsumexp(liws))    

    cbkrnl, cbl = kernel_opt(
        iws=iws.astype(np.float32), 
        stats=trn_data[1].reshape(-1,1).astype(np.float32),
        obs=obs_stats.reshape(1,1).astype(np.float32), 
        kernel_loss='x_kl', 
        epochs=10000, #epochs 
        minibatch=1000, #minibatch, 
        stop_on_nan=False,
        seed=42, 
        monitor=None)

    tau_est = 1./cbkrnl.B

    prop_prior_marg_samples, marg_iws = compute_marginals(g.proposal, g.prior)
    cbk = cbkrnl.eval(prop_prior_marg_samples)
    marg_kiws = marg_iws * cbk
    
    prop_prior_marg_density = KernelDensity(kernel='gaussian', 
                                           bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prop_prior_marg_samples)
    iw_marg_density = KernelDensity(kernel='gaussian', 
                                    bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prop_prior_marg_samples, 
                                                                                sample_weight=marg_iws)
    ax = plt.subplot(gs[0,0])
    ax.plot(xs, np.exp(iw_marg_density.score_samples(xs.reshape(-1,1))),  color=clrs[i], 
            label='importance weighted', linewidth=linewidth)        
    ax = plt.subplot(gs[0,0])
    ax.plot(xs, np.exp(prop_prior_marg_density.score_samples(xs.reshape(-1,1))), 
            c=clrs[i], alpha=0.5, label='proposal prior', linewidth=linewidth/2)
    
    
    kiw_marg_density = KernelDensity(kernel='gaussian', 
                                    bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prop_prior_marg_samples, 
                                                                                sample_weight=marg_kiws)
    ax = plt.subplot(gs[0,1])
    ax.plot(xs, np.exp(kiw_marg_density.score_samples(xs.reshape(-1,1))),  color=clrs[i], 
            label='importance weighted', linewidth=linewidth)    
    ax = plt.subplot(gs[0,1])
    ax.plot(xs, np.exp(prop_prior_marg_density.score_samples(xs.reshape(-1,1))), 
            c=clrs[i], alpha=0.5, label='proposal prior', linewidth=linewidth/2)
    
    
for i in range(2):
    ax = plt.subplot(gs[0,i])
    prior_marg_samples = compute_marginals(g.prior)
    prior_marg_density = KernelDensity(kernel='gaussian', 
                                       bandwidth=0.05 * (xlims[1] - xlims[0])).fit(prior_marg_samples)
    ax.plot(xs, np.exp(prior_marg_density.score_samples(xs.reshape(-1,1))), '--', c='b', label='prior', linewidth=3)
    ax.axvline(xobs, lw=2, c='k')
    plt.ylabel('density', fontsize=fontsize)
    ax.set_xlabel("$x$", fontsize=fontsize)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    #plt.yticks([])
    plt.xticks([xobs], [r'$x_{o}$'], fontsize=fontsize)
    ax.set_xlim(xlims)
    #ax.set_ylim([0, ])


plt.tight_layout()

plt.savefig(path + 'recovered_locality.pdf')

plt.show()